In [1]:
import pandas as pd
from datetime import datetime
import requests

In [2]:
start_date = '2024-05-24'
end_date = '2024-05-25'

In [3]:
url = f'https://api.energy-charts.info/public_power?country=si&start={start_date}&end={end_date}'
response = requests.get(url)

data = response.json()

In [4]:
print('Number of price items:', len(data['production_types'][0]['data']))
print('Number of unix time items:', len(data['unix_seconds']))

if len(data['production_types'][0]['data']) == len(data['unix_seconds']):
    print('Lengths are matching the expected values')

In [5]:
production_data = {}
for i in range(len(data['production_types'])):
    print(data['production_types'][i]['name'])
    production_per_type = []
    for j in range(len(data['production_types'][i]['data'])):
        production = data['production_types'][i]['data'][j]
        timestamp = datetime.fromtimestamp(data['unix_seconds'][j]).strftime('%Y-%m-%dT%H:%M:%S')
        
        production_per_type.append({'date': timestamp, 'production': production})
        
    production_data[data['production_types'][i]['name']] = production_per_type

In [6]:
dfs = []
for key in production_data.keys():
    print(key)
    df = pd.DataFrame(production_data[key])
    df['date'] = pd.to_datetime(df['date'])
    dfs.append(df)

In [7]:
dfs[0].head()

In [8]:
weather_url = f'https://archive-api.open-meteo.com/v1/archive?latitude=46.0833&longitude=15&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,cloud_cover,wind_speed_10m&timezone=Europe%2FBerlin'
response = requests.get(weather_url)

weather_data = response.json()

In [9]:
daily_weather_data = []
for i in range(len(weather_data['hourly']['time'])):
    temperature = weather_data['hourly']['temperature_2m'][i]
    humidity = weather_data['hourly']['relative_humidity_2m'][i]
    precipitation = weather_data['hourly']['precipitation'][i]
    cloud_cover = weather_data['hourly']['cloud_cover'][i]
    wind_speed = weather_data['hourly']['wind_speed_10m'][i]
    
    new_data = {'date': weather_data['hourly']['time'][i], 'temperature': temperature, 'humidity': humidity, 'precipitation': precipitation, 'cloud_cover': cloud_cover, 'wind_speed': wind_speed}
    daily_weather_data.append(new_data)

In [10]:
df_weather = pd.DataFrame(daily_weather_data)
df_weather.head(48)

In [11]:
print(df_weather.shape)
for df in dfs: print(df.shape)

In [12]:
for df in dfs: print(df.dtypes)

In [13]:
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather.dtypes

In [14]:
for df in dfs: print(df.isnull().sum())

In [15]:
df_weather.isnull().sum()

In [16]:
production_types = list(production_data.keys())
production_types_modified = [ptype.lower().replace(' ', '_').replace('/', '_') for ptype in production_types]

for i, df in enumerate(dfs):
    df_merged = pd.merge(df, df_weather, on='date')
    print(df_merged.head())
    df_merged.to_csv(f'../data/processed/{production_types_modified[i]}.csv', index=False)